<a href="https://colab.research.google.com/github/cr2uchile/Quality_Control_FireScarCL/blob/main/2_generate_csv_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing FireScar related statistics

Following we have code to test the contents of the FireScar-CL database, by reading 4 generated csv's: 
* that contain paths, filename, filesize and last modification date
* that contains tif and shapefile details per fire
* master file that contains all fires, and more information for the ones that were evaluated
* conaf revised XX that contains the base database YY

The tests are performed in several areas: 
* 1) Amount of files, Types of Files, Content of tif and shapefiles
* Geographic revision
* Date and season revision
* Burned area revision

.. 


para que sepas cuáles son: 
folderSeason	
Fire Scar_image_name	
PreF-image_name	
PostF-image_name	
RdNBR_images_name	
Severity_image_name	
Fire Scar_vector_name	
Severity_vector_name	fireId	
Treshold	
tif1Geo0	 lat top 
tif1Geo3	 lon left
tif1Geo1	 lat bottom 
tif1Geo5	 lon right
x
y
columnsShape1	
columnsShape2



## Imports, Loading
Mounts the Drive to read the summary csv files

In [ ]:
import sys
import os
import csv
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ogr
import unittest
#import pandas.testing
from pandas.testing import assert_frame_equal
from pandas.testing import assert_series_equal
from pandas.testing import assert_index_equal


In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 
#listOfFiles = os.listdir('drive/MyDrive')



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
sys.path.insert(0,'/content/drive/Shareddrives/CR2_Carpetas_Compartidas/06. Proyectos internos/Incendios_BD_Cicatrices/Proyecto_FireScarHistórico/ColabNotebooks/')
from common import *

print (output_path)
print (conaf_filename)


drive/Shared drives/CR2_Carpetas_Compartidas/06. Proyectos internos/Incendios_BD_Cicatrices/Proyecto_FireScarHistórico/Report/
RegistroIncendiosCONAF_depurada_Final.csv


In [ ]:
# Location Shapefile settings
ISOcode_region = {
    'Arica y Parinacota':'CL-AP',
    'Tarapaca':'CL-TA',
    'Antofagasta':'CL-AN',
    'Magallanes':'',
    'Aysen':'',
    'Atacama':'CL-AT',
    'Coquimbo':'CL-CO',
    'Valparaiso':'CL-VS',
    'Metropolitana':'CL-RM',
    'Los Lagos':'CL-LL',
    'Los Rios':'CL-LR',
    'Araucania':'CL-AR',
    'BioBio':'CL-BI',
    'Nuble':'CL-NB',
    'Maule':'CL-ML',
    'Ohiggins':'CL-LI',
}

code_region = {
    'Región de Arica y Parinacota' : 'Arica y Parinacota',
    'Región de Tarapacá' : 'Tarapaca',
    'Región de Antofagasta' : 'Antofagasta',
    'Región de Magallanes y Antártica Chilena': 'Magallanes',
    'Región de Aysén del Gral.Ibañez del Campo': 'Aysen',
    'Región de Atacama': 'Atacama',
    'Región de Coquimbo': 'Coquimbo',
    'Región de Valparaíso': 'Valparaiso',
    'Región Metropolitana de Santiago': 'Metropolitana',
    'Región de Los Lagos': 'Los Lagos',
    'Región de Los Ríos': 'Los Rios',
    'Región de La Araucanía': 'Araucania',
    'Región del Bío-Bío': 'BioBio',
    'Región de Ñuble': 'Nuble',
    'Región del Maule': 'Maule',
    "Región del Libertador Bernardo O'Higgins": 'Ohiggins',
}

drv = ogr.GetDriverByName('ESRI Shapefile') #We will load a shape file
regionshp = base_path+'ColabNotebooks/location/Regional.shp'
region_source = drv.Open(regionshp)
if region_source is None:
  print('Could not open ' ,regionshp)
rlayer = region_source.GetLayer(0)
idx_reg = rlayer.GetLayerDefn().GetFieldIndex("Region")   # CODE IS objectid  / 
#If the latitude/longitude we're going to use is not in the projection
#of the shapefile, then we will get erroneous results.
#The following assumes that the latitude longitude is in WGS84
#This is identified by the number "4326", as in "EPSG:4326"
#We will create a transformation between this and the shapefile's
#project, whatever it may be
rgeo_ref = rlayer.GetSpatialRef()
rpoint_ref=ogr.osr.SpatialReference()
rpoint_ref.ImportFromEPSG(4326)
rctran=ogr.osr.CoordinateTransformation(rpoint_ref,rgeo_ref)
#print(rctran)

def getLocation(layer, inlon, inlat):
  #Transform incoming longitude/latitude to the shapefile's projection
  # print(inlon, inlat)
  [lon,lat,z]=rctran.TransformPoint(inlon,inlat)
  #Create a point
  pt = ogr.Geometry(ogr.wkbPoint)
  pt.SetPoint_2D(0, lon, lat)
  #Set up a spatial filter such that the only features we see when we
  #in "layer" is the which overlap the point defined above
  layer.SetSpatialFilter(pt)

  if len(layer) == 1:
    return code_region.get(layer.GetNextFeature().GetFieldAsString(idx_reg))
  elif len(layer) > 1:
    return "Multiple Layers"
  else:    
    print("not found") 
    if inlon> -72 and inlat<-39:
      #return "^"+getLocation(layer, inlon-0.1, inlat)
      return getLocation(layer, inlon-0.1, inlat)
    else:
      #return "*"+getLocation(layer, inlon+0.1, inlat)
      return getLocation(layer, inlon+0.1, inlat)
#print(getLocation(rlayer, -71.79375,-36.28001))
print(getLocation(rlayer, -71.5176	, -32.1289))
print(getLocation(rlayer, -71.4214	, -39.0033))
print(getLocation(rlayer, -73.1143	, -36.6145))
print(getLocation(rlayer, -73.1541	, -36.8831))
print(getLocation(rlayer, -70.965421	, -34.286302))

# -71.4214 -39.0033


not found
Coquimbo
not found
Araucania
not found
not found
BioBio
not found
BioBio
Ohiggins


In [ ]:
def get_season(fireDate):
  #print (fireDate)
  year=int(fireDate[:4])  
  month=int(fireDate[4:6])  
  # print (year, month)
  if month >= 7:
    return str(year+1)
  return str(year)

print(get_season('201606'))
print(get_season('201607'))

def get_ISOCode(region):
  return ISOcode_region.get(region)

print (get_ISOCode('Nuble'))
print (get_ISOCode('Metropolitana'))


2016
2017
CL-NB
CL-RM


## Settings and Filenames
Loads and calculates the most important and general dataframes

In [ ]:
dfs_filename=os.path.join(output_path,'summary_20211116_VF.csv')
dfd_filename=os.path.join(output_path,'report_content_20211001D.csv')
dfm_filename=os.path.join(base_path,'Maestro_Evaluados_27_09_2021_NAMESOK.csv')
dfm2_filename=os.path.join(base_path,'PANGAEA','2021_10_12_FireScarCL_records_V2.csv')
dfi_filename= os.path.join(base_path,'PANGAEA','intersect_fireids_fixed.csv')
dfconaf_filename = base_path+conaf_filename

In [ ]:
# Reads intersect file
dfi = pd.read_csv(dfi_filename, dtype=str)
# KEYID,interaccion
dfi.rename(columns={'KEYID':'fireId'}, inplace=True)
dfi.set_index('fireId', inplace=True)
dfi.fillna('', inplace=True)
dfi.sort_index(inplace=True)

print(dfi.head)
print('len dfi', len(dfi))


<bound method NDFrame.head of          interaccion
fireId              
ID100000            
ID10002             
ID100027            
ID100032            
ID100041            
...              ...
ID99979     ID100150
ID9998              
ID99985             
ID99995             
ID99998             

[8153 rows x 1 columns]>
len dfi 8153


In [ ]:
# Reads summary file
dfs = pd.read_csv(dfs_filename, dtype=str)
print(dfs['folderR_2'].unique())

dfs['fileSize'] = dfs['fileSize'].astype(int)
dfs['folderR_2'] = dfs['folderR_2'].str.replace('Los','Los ')
dfs['folderS_2'] = dfs['folderS_2'].str.replace('Los','Los ')
print(dfs['folderR_2'].unique())
print(dfs['folderS_2'].unique())

dfregions = dfs.groupby(['folderRegion'], group_keys=False, as_index=False)['fileSize'].agg(['count', 'sum',  'mean', 'median', 'min', 'max']).reset_index()
dfregions['fireCount'] = dfregions['count']/17
dfseasons = dfs.groupby(['folderRegion','folderSeason'], group_keys=False, as_index=False)['fileSize'].agg(['count', 'sum',  'mean', 'median','min',  'max']).reset_index() 
dffires = dfs.groupby(['folderRegion','folderR_0', 'folderR_1', 'folderR_2', 'folderSeason', 'folderS_0','folderS_1','folderS_2', 'folderS_3', 'fireId', 'fireRegion', 'fireThreshold', 'fireDate'] , group_keys=False, as_index=False)['fileSize'].agg(['count', 'sum',  'mean', 'median','min',  'max']).reset_index()
# SET INDEX
dffires.set_index('fireId', inplace=True)

print('len dfs', len(dfs))
print('len fires', len(dffires.index))
dffires.sort_index(inplace=True)
print(dffires.columns)


['Coquimbo' 'Araucania' 'BioBio' 'LosLagos' 'LosRios' 'Maule'
 'Metropolitana' 'Valparaiso' 'Ohiggins']
['Coquimbo' 'Araucania' 'BioBio' 'Los Lagos' 'Los Rios' 'Maule'
 'Metropolitana' 'Valparaiso' 'Ohiggins']
['Coquimbo' 'Araucania' 'BioBio' 'Los Lagos' 'Los Rios' 'Maule'
 'Metropolitana' 'Valparaiso' 'Ohiggins']
len dfs 138601
len fires 8153
Index(['folderRegion', 'folderR_0', 'folderR_1', 'folderR_2', 'folderSeason',
       'folderS_0', 'folderS_1', 'folderS_2', 'folderS_3', 'fireRegion',
       'fireThreshold', 'fireDate', 'count', 'sum', 'mean', 'median', 'min',
       'max'],
      dtype='object')


In [ ]:
# Reads details file
dfd = pd.read_csv(dfd_filename, index_col='fireId')

print(dfd.columns)
print('len dfdetails - dfd', len(dfd))
dfd['IdKey'] = 'ID'+dfd['KEY'].astype(str)

dfd['Temporada'] = dfd['Temporada'].astype(str)
dfd['Temporada.1'] = dfd['Temporada.1'].astype(str)

dfd.sort_index(inplace=True)
print(dfd['Temporada'].dtypes)
#print(dfd.head)

Index(['tif1Meta', 'tif1Proj', 'tif1Geo0', 'tif1Geo3', 'tif1Geo1', 'tif1Geo5',
       'tif1RCount', 'tif1X', 'tif1Y', 'tif1Zeros', 'tif1Ones', 'tif1Min',
       'tif1Max', 'tif2Geo0', 'tif2Geo3', 'tif2Geo1', 'tif2Geo5', 'tif2X',
       'tif2Y', 'tif2Zeros', 'tif2Ones', 'tif2Twos', 'tif2Threes', 'tif2Min',
       'tif2Max', 'tif3Geo0', 'tif3Geo3', 'tif3Geo1', 'tif3Geo5', 'tif3X',
       'tif3Y', 'tif3Min', 'tif3Max', 'tif4Geo0', 'tif4Geo3', 'tif4Geo1',
       'tif4Geo5', 'tif4RCount', 'tif4X', 'tif4Y', 'tif5Geo0', 'tif5Geo3',
       'tif5Geo1', 'tif5Geo5', 'tif5X', 'tif5Y', 'columnsShape1',
       'lencolumnsShape1', 'lenShape1', 'columnsShape2', 'lencolumnsShape2',
       'lenShape2', 'NIR_min', 'AreaTotal', 'Temporada', 'AreaINC_ha',
       'AreaUnchS', 'AreaLowS', 'AreaHighS', 'Control', 'AreaMedS', 'Inicio',
       'Region', 'NDVI_mean', 'Nombre_Inc', 'Area_Poly', 'KEY', 'AreaTotal.1',
       'Temporada.1', 'AreaINC_ha.1', 'Severity', 'AreaUnchS.1', 'AreaLowS.1',
       'AreaHighS.1

In [ ]:
# Read original master file
dfm = pd.read_csv(dfm_filename, dtype=str)
dfm['fireId'] = 'ID'+dfm['KEY']
dfm.loc[:,['lat', 'long']] = dfm[['lat', 'long']].astype(float)
dfm['valid_start_date'] = dfm['Start_year']+'-'+dfm['Start_month'].str.zfill(2)+'-'+dfm['Start_day'].str.zfill(2)
dfm['stripped_start_date'] = dfm['Start_year']+dfm['Start_month'].str.zfill(2)+dfm['Start_day'].str.zfill(2)
dfm['valid_control_date'] = dfm['Control_year']+'-'+dfm['Control_month'].str.zfill(2)+'-'+dfm['Control_day'].str.zfill(2)
dfm.set_index('fireId', inplace=True)
dfm.sort_index(inplace=True)
dfmeval = dfm.loc[((dfm.Obtained == 'Si') & (dfm.Revision != 'No')) | (dfm.Revision == 'Si')  ]   
#print(dfm.columns)
print('len dfmaster - dfm', len(dfm))
#print(dfm.head)
print('len dfmeval', len(dfmeval))
print(dfmeval.index)

len dfmaster - dfm 12250
len dfmeval 8153
Index(['ID100000', 'ID10002', 'ID100027', 'ID100032', 'ID100041', 'ID100057',
       'ID100058', 'ID10006', 'ID10007', 'ID10008',
       ...
       'ID9994', 'ID99948', 'ID99957', 'ID99962', 'ID99978', 'ID99979',
       'ID9998', 'ID99985', 'ID99995', 'ID99998'],
      dtype='object', name='fireId', length=8153)


In [ ]:
# Read regenerated master file
dfm2 = pd.read_csv(dfm2_filename, dtype=str)
dfm2['fireId'] = 'ID'+dfm2['ID']
dfm2.loc[:,['Latitude', 'Longitude']] = dfm2[['Latitude', 'Longitude']].astype(float)
dfm2.set_index('fireId', inplace=True)
dfm2['RegionCode'] = dfm2['Region_CONAF'].apply(get_ISOCode)

dfm2.sort_index(inplace=True)
print(dfm2.columns)
print('len dfmaster - dfm', len(dfm2))
dfm2eval = dfm2.loc[(dfm2.FireScar == '1')]   
print('len dfm2eval', len(dfm2eval))

#print(dfm2.head)
#print(dfm2.index)

Index(['ID', 'FireSeason', 'Region_CONAF', 'FireName_CONAF', 'Area_CONAF',
       'IgnitionDate_CONAF', 'ControlDate_CONAF', 'Longitude', 'Latitude',
       'UTM_X', 'UTM_Y', 'FireScar', 'RegionCode'],
      dtype='object')
len dfmaster - dfm 12250
len dfm2eval 8153


In [ ]:
# Read CONAF file
dfconaf = pd.read_csv(dfconaf_filename, dtype=str)
dfconaf['fireId'] = 'ID'+dfconaf['KEY']
dfconaf['valid_start_date'] = dfconaf['inicioano']+'-'+dfconaf['iniciomes'].str.zfill(2)+'-'+dfconaf['iniciodia'].str.zfill(2)
dfconaf['valid_control_date'] = dfconaf['controlano']+'-'+dfconaf['controlmes'].str.zfill(2)+'-'+dfconaf['controldia'].str.zfill(2)
dfconaf.set_index('fireId', inplace=True)
dfconaf.loc[:,['lat', 'long']] = dfconaf[['lat', 'long']].astype(float)
dfconaf.sort_index(inplace=True)
print('len conaf', len(dfconaf))
#print(dfconaf.columns)
#print(dfconaf.head)
dfconafeval = dfconaf[dfconaf.index.isin(dfd.index)]
#print(dfconafeval.index)

len conaf 195358


In [ ]:
# Generates Locations MERGE
dflocations=pd.merge(dfconafeval[[ 'Region', 'Cod_Region', 'Nombre inc', 'long', 'lat']], 
                     dfd[['tif1Geo0', 'tif1Geo3', 'tif1Geo1', 'tif1Geo5', 'tif1X', 'tif1Y', 'Region', 'Region.1']],
                     on='fireId', suffixes=['_conaf', '_dbf']).merge(dffires[['folderRegion', 'folderR_0', 'folderR_1', 'folderR_2', 'folderSeason', 'folderS_0', 'folderS_1', 'folderS_2', 'folderS_3', 'fireRegion', 'fireThreshold', 'fireDate']], on='fireId')

dflocations['season_from_date'] = dflocations['fireDate'].apply(get_season)

print(dflocations.columns)
print(dflocations.Region_dbf)
#print(dflocations.dtypes)

Index(['Region_conaf', 'Cod_Region', 'Nombre inc', 'long', 'lat', 'tif1Geo0',
       'tif1Geo3', 'tif1Geo1', 'tif1Geo5', 'tif1X', 'tif1Y', 'Region_dbf',
       'Region.1', 'folderRegion', 'folderR_0', 'folderR_1', 'folderR_2',
       'folderSeason', 'folderS_0', 'folderS_1', 'folderS_2', 'folderS_3',
       'fireRegion', 'fireThreshold', 'fireDate', 'season_from_date'],
      dtype='object')
fireId
ID100000     Maule
ID10002     BioBio
ID100027     Maule
ID100032     Maule
ID100041     Maule
             ...  
ID99979      Maule
ID9998      BioBio
ID99985      Maule
ID99995      Maule
ID99998      Maule
Name: Region_dbf, Length: 8153, dtype: object


In [ ]:
dflocations['obtainedRegion'] = dflocations.apply(lambda x: getLocation(rlayer, x['long'], x['lat']), axis=1)
# 6 min
print(dflocations.head)


not found
not found
not found
not found
not found
not found
not found
not found
not found
not found
<bound method NDFrame.head of          Region_conaf Cod_Region  ... season_from_date obtainedRegion
fireId                            ...                                
ID100000        Maule          7  ...             2003          Maule
ID10002        BioBio          8  ...             1986          Nuble
ID100027        Maule          7  ...             2003          Maule
ID100032        Maule          7  ...             2003          Maule
ID100041        Maule          7  ...             2003          Maule
...               ...        ...  ...              ...            ...
ID99979         Maule          7  ...             2003          Maule
ID9998         BioBio          8  ...             1986          Nuble
ID99985         Maule          7  ...             2003          Maule
ID99995         Maule          7  ...             2003          Maule
ID99998         Maule         

In [ ]:
dflocations['obtainedRegionCode'] = dflocations['obtainedRegion'].apply(get_ISOCode)

dflocations['observationRegions'] = np.where(dflocations['Region_dbf'] != dflocations['obtainedRegion'], 'The ignition point is located in the '+ dflocations['obtainedRegionCode'] +' region','')

dflocations['observationRegions'] = np.where(dflocations['obtainedRegion'] == 'Nuble', 'The ignition point is located in the current CL-NB region', dflocations['observationRegions'])



In [ ]:
print(dflocations.head)

<bound method NDFrame.head of          Region_conaf  ...                                 observationRegions
fireId                 ...                                                   
ID100000        Maule  ...                                                   
ID10002        BioBio  ...  The ignition point is located in the current C...
ID100027        Maule  ...                                                   
ID100032        Maule  ...                                                   
ID100041        Maule  ...                                                   
...               ...  ...                                                ...
ID99979         Maule  ...                                                   
ID9998         BioBio  ...  The ignition point is located in the current C...
ID99985         Maule  ...                                                   
ID99995         Maule  ...                                                   
ID99998         Maule  ...        

In [ ]:
# Generates MASTER MERGE

# REF dffires = dfs.groupby(['folderRegion','folderR_0', 'folderR_1', 'folderR_2', 'folderSeason', 'folderS_0','folderS_1','folderS_2', 'folderS_3', 'fireId', 
# REF dffires .. 'fireRegion', 'fireThreshold', 'fireDate'] , group_keys=False, as_index=False)['fileSize'].agg(['count', 'sum',  'mean', 'median','min',  'max']).reset_index()

# folderSeason	FireScar_image_name	PreF-image_name	PostF-image_name	RdNBR_images_name	Severity_image_name	Fire Scar_vector_name	Severity_vector_name	
# fireId	Treshold	tif1Geo0	tif1Geo3	tif1Geo1	tif1Geo5	firescar x	firescar y	['NIR_min', 'AreaTotal', 'Temporada', 'AreaINC_ha', 'AreaUnchS', 'AreaLowS', 'AreaHighS', 'Control', 'AreaMedS', 'Inicio', 'Region', 'NDVI_mean', 'Nombre_Inc', 'Area_Poly', 'KEY']	columnsShape2

# OK: folderLocation  G:FireScarImgName  G:PreFireImgName  G:PostFireImgName  G:RdNBRImgName G:SeverityImgName G:FireScarVectorName G:SeverityVectorName 
# OK: FireID 
# OK: Threshold 
# OK: FireScarUpperLat    G: FireScarLowerLat     OK: FireScarLeftLon      G: FireScarRightLon      OK: FireScarX  OK: FireScarY 
#### PreFireUpperLat PreFireLowerLat PreFireLeftLon PreFireRightLon PreFireX  PreFireY 
# FireScarPolygons, 
# FS_FireID, FS_Region, FS_Season, FS_IgnDate, FS_IgnLat, FS_IgnLon, FS_NdviMean, FS_PolygArea, FS_NirMin, FS_TotalArea, FS_AreaUnchS, FS_AreaLowS, FS_AreaModS, 
# FS_AreaHighS, FS_CtrlDate, FS_AreaCONAF, FS_NameCONAF, FS_OverlapIDs
# SeverityPolygons,
# S_FireID, S_Region, S_Season, S_IgnDate, S_IgnLat, S_IgnLon, S_Severity, S_TotalArea, S_AreaUnchS, S_AreaLowS, S_AreaModS, S_AreaHighS, S_CtrlDate, S_AreaCONAF, S_NameCONAF

dfmasterOut=pd.merge(dffires[['folderSeason', 'fireThreshold',  'folderS_1', 'fireDate']],
                     dfd[['tif1Geo0', 'tif1Geo3', 'tif1Geo1', 'tif1Geo5', 'tif1X', 'tif1Y', 
                          'AreaTotal', 'AreaUnchS', 'AreaLowS', 'AreaHighS', 'AreaMedS', 
                          'lenShape1', 'lenShape2' ]], 
                     on='fireId').merge(dfmeval[['valid_start_date', 'valid_control_date', 'Season_CONAF', 'Region_CONAF', 'Fire_Name_CONAF', 'Area_Ha_CONAF', 
                                                 'lat', 'long' ]], 
                                                 on='fireId').merge(dfi[['interaccion']], on='fireId').merge(dflocations[['observationRegions']], on='fireId')
# Generates Filenames, other corner, ... 
# tif1Geo0 = FireScarLeftLon tif1Geo0 (left lon)
# tif1Geo3 = FireScarUpperLat  tif1Geo3 (top lat) 
dfmasterOut['FireID']=dfmasterOut.index
dfmasterOut.rename(columns={'folderSeason':'FolderLocation', 'fireThreshold':'Threshold', 'tif1Geo3':'NorthBoundLatitude', 'tif1Geo0': 'WestBoundLongitude',
                            'tif1X':'Width', 'tif1Y':'Height', 'lenShape1':'FireScarPolygons', 'lenShape2':'SeverityPolygons',
                            'AreaTotal':'TotalArea', 'AreaMedS':'AreaModS'}, inplace=True)

dfmasterOut.rename(columns={'Season_CONAF':'FireSeason', 'Region_CONAF':'Region_CONAF', 'Fire_Name_CONAF':'FireName_CONAF', 'Area_Ha_CONAF':'Area_CONAF', 
                            'valid_start_date':'IgnitionDate', 
                            'valid_control_date':'ControlDate', 'long':'Longitude', 'lat':'Latitude',
                            'interaccion':'OverlapIDs', 'observationRegions':'Observations'}, inplace=True)

dfmasterOut['_commonName']='_'+dfmasterOut['folderS_1']+'_'+dfmasterOut.index+'_'+dfmasterOut['Threshold']+'_'+dfmasterOut['fireDate']
dfmasterOut['FireScarImgName'] = 'FireScar'+dfmasterOut['_commonName'] +'.tif'
dfmasterOut['PreFireImgName'] = 'ImgPreF'+dfmasterOut['_commonName'] +'.tif'
dfmasterOut['PostFireImgName'] = 'ImgPosF'+dfmasterOut['_commonName'] +'.tif'
dfmasterOut['RdNBRImgName'] = 'RdNBR'+dfmasterOut['_commonName'] +'.tif'
dfmasterOut['SeverityImgName'] = 'Severity'+dfmasterOut['_commonName'] +'.tif'
dfmasterOut['FireScarVectorName'] = 'FireScar'+dfmasterOut['_commonName'] +'.shp'
dfmasterOut['SeverityVectorName'] = 'Severity'+dfmasterOut['_commonName'] +'.shp'
dfmasterOut['Threshold'] = dfmasterOut['Threshold'].str[1:]
#print(dfmasterOut['Threshold'])
dfmasterOut['SouthBoundLatitude'] = dfmasterOut['NorthBoundLatitude'] + dfmasterOut['tif1Geo5'] * dfmasterOut['Height']
dfmasterOut['EastBoundLongitude'] = dfmasterOut['WestBoundLongitude'] + dfmasterOut['tif1Geo1'] * dfmasterOut['Width']


masterOutColumns = ['FireID', 'FireSeason', 'RegionCode', 'Region_CONAF', 'FireName_CONAF', 'Area_CONAF', 'IgnitionDate', 'ControlDate', 
                    'Longitude', 'Latitude', 
                    'FolderLocation', 'FireScarImgName', 'PreFireImgName', 'PostFireImgName', 'RdNBRImgName', 'SeverityImgName', 
                    'FireScarVectorName', 'SeverityVectorName',  'Threshold', 'NorthBoundLatitude', 'SouthBoundLatitude', 'WestBoundLongitude',
                    'EastBoundLongitude', 'Width', 'Height', 
                    'TotalArea', 'AreaUnchS', 'AreaLowS', 'AreaModS', 'AreaHighS',                            
                    'FireScarPolygons', 'SeverityPolygons', 'OverlapIDs', 'Observations']

dfm2['FireID']=dfm2.index

#print(dfmasterOut.columns)
#print(dfmasterOut.head)
#print(dfmasterOut.dtypes)

supermaster = pd.merge(dfm2, dfmasterOut[['FolderLocation', 'FireScarImgName', 'PreFireImgName', 'PostFireImgName', 'RdNBRImgName', 'SeverityImgName', 
                    'FireScarVectorName', 'SeverityVectorName',  'Threshold', 'NorthBoundLatitude', 'SouthBoundLatitude', 'WestBoundLongitude',
                    'EastBoundLongitude', 'Width', 'Height', 'TotalArea', 'AreaUnchS', 'AreaLowS', 
                    'AreaHighS', 'AreaModS', 'FireScarPolygons', 'SeverityPolygons', 'OverlapIDs', 'Observations']], how='left', on='fireId')


supermaster[['Threshold', 'NorthBoundLatitude', 'SouthBoundLatitude', 'WestBoundLongitude', 'EastBoundLongitude', 'Width', 'Height', 'FireScarPolygons', 'SeverityPolygons', 
             'TotalArea', 'AreaUnchS', 'AreaLowS', 'AreaModS', 'AreaHighS',
    'FireSeason', 'Area_CONAF', 'Longitude', 'Latitude' ]] = supermaster[['Threshold', 'NorthBoundLatitude', 'SouthBoundLatitude', 'WestBoundLongitude', 'EastBoundLongitude', 
                                                                          'Width', 'Height', 'FireScarPolygons', 'SeverityPolygons', 
                                                                          'TotalArea', 'AreaUnchS', 'AreaLowS', 'AreaModS', 'AreaHighS',
    'FireSeason', 'Area_CONAF', 'Longitude', 'Latitude' ]].apply(pd.to_numeric)

processing_text='Because of a processing capacity issue in Google Earth Engine, the vectorial version was develop in Qgis.'

#supermaster.loc['ID189174', 'Observations'] = processing_text
#supermaster.loc['ID189054', 'Observations'] = processing_text
#supermaster.loc['ID189255', 'Observations'] = processing_text
#supermaster.loc['ID189277', 'Observations'] = processing_text

print(supermaster['Observations']['ID189174'])
print(supermaster['Observations']['ID189054'])
print(supermaster['Observations']['ID189255'])
print(supermaster['Observations']['ID189277'])

# En estos incendios ID189174, ID189054, ID189255, ID189277  pondría:
#"Because of a processing capacity issue in Google Earth Engine, the vectorial version was develop in Qgis"
#
#
#mylat = dfsm['Latitude']['ID8301']


# SET UNITS
# Area_CONAF [ha] Longitude [°] Latitude [°] NorthBoundLatitude [°] SouthBoundLatitude [°] WestBoundLongitude [°] EastBoundLongitude [°] 
# Width [pixels] Height [pixels] TotalArea [m2] AreaUnchS [m2] AreaLowS [m2] AreaModS [m2] reaHighS [m2]
#

supermaster.rename(columns={'Area_CONAF':'Area_CONAF [ha]', 'NorthBoundLatitude':'NorthBoundLatitude [°]', 'WestBoundLongitude': 'WestBoundLongitude [°]',
                            'SouthBoundLatitude': 'SouthBoundLatitude [°]', 'EastBoundLongitude': 'EastBoundLongitude [°]',
                            'Longitude':'Longitude [°]', 'Latitude':'Latitude [°]', 
                            'Width':'Width [pixels]', 'Height':'Height [pixels]', 'TotalArea':'TotalArea [m2]', 'AreaUnchS':'AreaUnchS [m2]',
                            'AreaLowS':'AreaLowS [m2]', 'AreaModS':'AreaModS [m2]', 'AreaHighS':'AreaHighS [m2]'}, inplace=True)

supermasterColumns = ['FireID', 'FireSeason', 'RegionCode', 'Region_CONAF', 'FireName_CONAF', 'Area_CONAF [ha]', 'IgnitionDate_CONAF', 'ControlDate_CONAF', 
                      'Latitude [°]', 'Longitude [°]', 
                      'FireScar', 'FolderLocation', 'FireScarImgName', 'PreFireImgName', 'PostFireImgName', 'RdNBRImgName', 'SeverityImgName', 
                    'FireScarVectorName', 'SeverityVectorName',  'Threshold', 'NorthBoundLatitude [°]', 'SouthBoundLatitude [°]', 'WestBoundLongitude [°]',
                    'EastBoundLongitude [°]', 'Width [pixels]', 'Height [pixels]', 
                    'TotalArea [m2]', 'AreaUnchS [m2]', 'AreaLowS [m2]', 'AreaModS [m2]', 'AreaHighS [m2]', 
                    'FireScarPolygons', 'SeverityPolygons', 'OverlapIDs', 'Observations']




##Print Reports

In [ ]:
 def print_reports():
  full_path = output_path+'output_amounts_'+tag+'.xlsx'
  with pd.ExcelWriter(full_path) as writer:  
    dfregions.to_excel(writer, sheet_name='SizePerRegion')
    dfseasons.to_excel(writer, sheet_name='SizePerSeason')
    dffires.to_excel(writer, sheet_name='SizePerFire')  

  full_path = output_path+'output_location_'+tag+'.xlsx'
  with pd.ExcelWriter(full_path) as writer:  
    dflocations.to_excel(writer, sheet_name='LocationsPerFire')


  #full_path = output_path+'output_MASTER_'+tag+'.xlsx'
  #with pd.ExcelWriter(full_path) as writer:  
  #  print (masterOutColumns)
  #  print (dfmasterOut.columns)
  #  dfmasterOut.to_excel(writer, sheet_name='Fire_Summary', columns=masterOutColumns , index=False)


  full_path = output_path+'output_SUPERMASTER_'+tag+'.xlsx'
  with pd.ExcelWriter(full_path) as writer:  
    print (supermasterColumns)
    print (supermaster.columns)
    dif_list = list(set(supermaster.columns) - set(supermasterColumns))
    print(dif_list)

    supermaster.to_excel(writer, sheet_name='Fire_Summary', columns=supermasterColumns , index=False)


  # REGION from filename, regionfolder, seasonfolder
  # GET REGION (and comuna?) from CONAF lat long / MAP compare region reported from CONAF
  def test_print_reports(self):
    '''Printing Location report if needed'''
    #df['Diff'] = np.where( df['folderS_3'] == df['fireDate'].apply(get_season) , '1', '0')




In [ ]:
flag_print_reports = 1

tag='20211116_V10'

if flag_print_reports:
  print_reports()


['FireID', 'FireSeason', 'RegionCode', 'Region_CONAF', 'FireName_CONAF', 'Area_CONAF [ha]', 'IgnitionDate_CONAF', 'ControlDate_CONAF', 'Latitude [°]', 'Longitude [°]', 'FireScar', 'FolderLocation', 'FireScarImgName', 'PreFireImgName', 'PostFireImgName', 'RdNBRImgName', 'SeverityImgName', 'FireScarVectorName', 'SeverityVectorName', 'Threshold', 'NorthBoundLatitude [°]', 'SouthBoundLatitude [°]', 'WestBoundLongitude [°]', 'EastBoundLongitude [°]', 'Width [pixels]', 'Height [pixels]', 'TotalArea [m2]', 'AreaUnchS [m2]', 'AreaLowS [m2]', 'AreaModS [m2]', 'AreaHighS [m2]', 'FireScarPolygons', 'SeverityPolygons', 'OverlapIDs', 'Observations']
Index(['ID', 'FireSeason', 'Region_CONAF', 'FireName_CONAF', 'Area_CONAF [ha]',
       'IgnitionDate_CONAF', 'ControlDate_CONAF', 'Longitude [°]',
       'Latitude [°]', 'UTM_X', 'UTM_Y', 'FireScar', 'RegionCode', 'FireID',
       'FolderLocation', 'FireScarImgName', 'PreFireImgName',
       'PostFireImgName', 'RdNBRImgName', 'SeverityImgName',
       '

## GENERATE ZIP FILES

In [ ]:
# GENERATE ZIP FILES 
###!zip "/content/drive/path/output_file_name.zip" "/content/drive/My Drive/Tutorial/games"
zipfile=os.path.join(base_path,'Files_PANGAEA', 'FireScar_CL-CO_Coquimbo_1988-2018.zip')
infolder=os.path.join(base_path,'Fire_CL-CO_Coquimbo')
print(zipfile)
print(infolder)

sys.path.insert(0,'/content/drive/Shareddrives/CR2_Carpetas_Compartidas/06. Proyectos internos/Incendios_BD_Cicatrices/')
!ls /content/drive/Shared\ drives/CR2_Carpetas_Compartidas/

# COQUIMBO 
!zip -r /content/drive/Shared\ drives/CR2_Carpetas_Compartidas/06.\ Proyectos\ internos/Incendios_BD_Cicatrices/Proyecto_FireScarHistórico/Files_PANGAEA/FireScar_CL-CO_Coquimbo_1988-2018.zip Fire_CL-CO_Coquimbo/


#!zip -r /content/drive/Shared\ drives/CR2_Carpetas_Compartidas/06.\ Proyectos\ internos/Incendios_BD_Cicatrices/Proyecto_FireScarHistórico/Files_PANGAEA/FireScar_CL-CO_Coquimbo_1988-2018.zip Fire_CL-CO_Coquimbo/



#dfs_filename=os.path.join(output_path,'summary_20211116_VF.csv')
#dfd_filename=os.path.join(output_path,'report_content_20211001D.csv')
#dfm_filename=os.path.join(base_path,'Maestro_Evaluados_27_09_2021_NAMESOK.csv')
#dfm2_filename=os.path.join(base_path,'PANGAEA','2021_10_12_FireScarCL_records_V2.csv')
#dfi_filename= os.path.join(base_path,'PANGAEA','intersect_fireids_fixed.csv')
#dfconaf_filename = base_path+conaf_filename

drive/Shared drives/CR2_Carpetas_Compartidas/06. Proyectos internos/Incendios_BD_Cicatrices/Proyecto_FireScarHistórico/Files_PANGAEA/FireScar_CL-CO_Coquimbo_1988-2018.zip
drive/Shared drives/CR2_Carpetas_Compartidas/06. Proyectos internos/Incendios_BD_Cicatrices/Proyecto_FireScarHistórico/Fire_CL-CO_Coquimbo
'00. Fotos, logos, gráficas'
'01. Minutas CR2-Gestion'
'02. Publicaciones'
'03. Convenios Honorarios'
'04. Inventario'
'05. Proyectos externos'
'06. Proyectos internos'
'07. Colaboracion Instituciones'
'08. Reuniones Ampliadas CR2'
'09. Talleres y Webinars CR2'
'10. Capacitaciones internas'
'11. Pack Bienvenida al CR2'
'13. Videos'
'14. Comunicaciones'
'15. RedLama'
'18. Webs CR2 ++'
'19. Llamados CR2'
'21. Staff CR2 (inv, postd, est)'
'22. Listas de contactos'
'23. Traducciones'
'24. Compromisos - Autoevaluaciones'
'25. Conferencias CR2'
 Admin
'Boya Posar'
'Coordinación interna CR2'
'CR2 HITOS 2020.gsheet'
'Escuela de Verano 2021'
'Iniciativas Docencia CR2'
'Propiedad Intelectu

## Class Generic Test

In [ ]:
class testDF(unittest.TestCase):
  def assertDataframeEqual(self, a, b, msg):
    try:
      assert_frame_equal(a, b,  check_index_type=False, check_dtype=False, check_names=False)      
    except AssertionError as e:
      raise self.failureException(msg) from e

  def assertIndexEqual(self, a, b, msg):
    #print("COMPARING INDEXES")
    try:
      assert_index_equal(a, b,  check_names=False)      
    except AssertionError as e:
      raise self.failureException(msg) from e

  def setUp(self):
    '''Setup common data and base dataframes for comparing emptyness'''
    self.addTypeEqualityFunc(pd.DataFrame, self.assertDataframeEqual)
    self.addTypeEqualityFunc(pd.Index, self.assertIndexEqual)

    self.emptyc=pd.DataFrame({'self': pd.Series(dtype='float64'), 'other': pd.Series(dtype='float64') })
    self.emptydfd=pd.DataFrame( {c:pd.Series(dtype='object') for c in dfd.columns})
    self.emptyidx = pd.Index([],  name='fireId')
    #@unittest.skip 

## Class Test File Amounts 
 Test amount of files by extension, type, total amount


In [ ]:
class testFileAmounts(testDF):
  def setUp(self):
    '''Set up dataframes that are used in several tests'''
    testDF.setUp(self)
    self.dfNumFilesType = dfs.groupby(['folderRegion','folderSeason', 'fireId', 'fireType'])['fireType'].count().reset_index(name="count")
    self.dfNumFilesExt = dfs.groupby(['folderRegion','folderSeason', 'fireId', 'fileExt'])['fileExt'].count().reset_index(name="count")

  def test_total_size(self):
    '''Test case total size'''
    totalSize = dfs['fileSize'].sum()
    print(totalSize) 
    self.assertTrue(totalSize > 120000000000)
    # 116GB  # 124828781471 

  def test_region_amounts(self):
    '''Test case amount of regions considered'''    
    self.assertEqual(len(dfregions), 9)
  
  def test_region_season_amount (self):
    '''Test case amount of regions-seasons considered'''            
    self.assertEqual(len(dfseasons), 244)

  def test_internal_fire_amount(self):
    '''Test case amount of fires considered, Detailed obtained Files vs Master obtained Fires'''   
    self.assertEqual(len(dfd), len(dfmeval))

  def test_total_fire_amount(self):
    '''Test case amount of fires considered, Summary obtained Files vs Master obtained Fires'''    
    self.assertEqual(len(dffires), len(dfmeval))

  def test_master_vs_files(self):
    '''Test Files in Masterlist Obtained and NOT Filesystem'''
    ridx = dfmeval.index.difference(dffires.index)
    self.assertEqual(ridx, self.emptyidx, ridx)

  def test_details_vs_files(self):
    '''Test Files in Details Obtained and NOT Filesystem'''
    ridx = dfd.index.difference(dffires.index)
    self.assertEqual(ridx, self.emptyidx, ridx)

  def test_details_vs_files2(self):
    '''Test Files in Details Obtained and NOT Master'''
    ridx = dfd.index.difference(dfmeval.index)
    self.assertEqual(ridx, self.emptyidx, ridx)    

  def  test_master_vs_files2(self):
    '''Test Files in Filesystem and NOT in Masterlist Obtained'''    
    ridx = dffires.index.difference(dfmeval.index)
    self.assertEqual(ridx, self.emptyidx, ridx)

  def  test_master_vs_files3(self):
    '''Test Files in Filesystem and NOT in Masterlist'''
    #rdf3 =  dffires[-dffires.isin(dfm)]
    ridx = dffires.index.difference(dfm.index)
    self.assertEqual(ridx, self.emptyidx, ridx)

  def  test_master_vs_details(self):
    '''Test Files in Filesystem and NOT in Details'''
    #rdf3 =  dffires[-dffires.isin(dfm)]
    ridx = dffires.index.difference(dfd.index)
    self.assertEqual(ridx, self.emptyidx, ridx)    

  def  test_master_vs_details2(self):
    '''Test Files in MASTER and NOT in Details'''
    ridx = dfmeval.index.difference(dfd.index)
    self.assertEqual(ridx, self.emptyidx, ridx)    

  def test_fire_files_amount(self):
    '''Test Number of Correct Files per Fire'''
    rdf4 = dffires.loc[dffires['count'] != 17]
    self.assertEqual (len(rdf4),0)

  def test_file_types1(self):
    '''Test Fires with wrong amount of FireScar files (!= 7)'''
    countdf = self.dfNumFilesType.loc[(self.dfNumFilesType['fireType'] == 'FireScar') & (self.dfNumFilesType['count'] != 7)]
    self.assertEqual(len(countdf),0) 

  def test_file_types2(self):
    '''Test Fires with wrong amount of Severity files (!= 7)'''
    countdf = self.dfNumFilesType.loc[(self.dfNumFilesType['fireType'] == 'Severity') & (self.dfNumFilesType['count'] != 7)]
    self.assertEqual(len(countdf),0)  

  def test_file_types3(self):
    '''Test Fires with wrong amount of ImgPosF files (!= 1)'''
    countdf = self.dfNumFilesType.loc[(self.dfNumFilesType['fireType'] == 'ImgPosF') & (self.dfNumFilesType['count'] != 1)]
    self.assertEqual(len(countdf),0)

  def test_file_types4(self):    
    '''Test Fires with wrong amount of ImgPreF (!= 1)'''
    countdf = self.dfNumFilesType.loc[(self.dfNumFilesType['fireType'] == 'ImgPreF') & (self.dfNumFilesType['count'] != 1)]
    self.assertEqual(len(countdf),0)

  def test_file_types5(self):
    '''Test Fires with wrong amount of RdNBR (!= 1)'''
    countdf = self.dfNumFilesType.loc[(self.dfNumFilesType['fireType'] == 'RdNBR') & (self.dfNumFilesType['count'] != 1)]    
    self.assertEqual(len(countdf),0)
  
  def test_file_ext1(self):
    '''Test Fires with wrong amount of tif  (!= 5)'''
    countdf = self.dfNumFilesExt.loc[(self.dfNumFilesExt['fileExt'] == '.tif') & (self.dfNumFilesExt['count'] != 5)]
    self.assertEqual(len(countdf),0)

  def test_file_ext2(self):
    '''Test Fires with wrong amount of shape related files  (!= 2)'''
    self.assertEqual(len(self.dfNumFilesExt.loc[(self.dfNumFilesExt['fileExt'] == '.cpg') & (self.dfNumFilesExt['count'] != 2)]),0)
    self.assertEqual(len(self.dfNumFilesExt.loc[(self.dfNumFilesExt['fileExt'] == '.dbf') & (self.dfNumFilesExt['count'] != 2)]),0)
    self.assertEqual(len(self.dfNumFilesExt.loc[(self.dfNumFilesExt['fileExt'] == '.fix') & (self.dfNumFilesExt['count'] != 2)]),0)
    self.assertEqual(len(self.dfNumFilesExt.loc[(self.dfNumFilesExt['fileExt'] == '.prj') & (self.dfNumFilesExt['count'] != 2)]),0)
    self.assertEqual(len(self.dfNumFilesExt.loc[(self.dfNumFilesExt['fileExt'] == '.shp') & (self.dfNumFilesExt['count'] != 2)]),0)
    self.assertEqual(len(self.dfNumFilesExt.loc[(self.dfNumFilesExt['fileExt'] == '.shx') & (self.dfNumFilesExt['count'] != 2)]),0)  






## Class Tiff Detailed Checks

Test tiff details: locations, content, xxx
Check TIFS rastercounts, zeros and ones, size, metadata in general


In [ ]:
class testTifDetails(testDF): 
   
  def test_projection(self):
    '''Test projection text''' 
    projection_text ='GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]'
    rdf = dfd.loc[dfd['tif1Proj'] != projection_text]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())
    
  def test_area(self):
    '''Test area text''' 
    area_text = "{'AREA_OR_POINT': 'Area'}"
    rdf = dfd.loc[dfd['tif1Meta'] != area_text]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_rastercount(self):
    '''Test Correct amount of rasters per tif'''
    self.assertEqual(len(dfd.loc[dfd['tif1RCount'] != 1]), 0)
    self.assertEqual(len(dfd.loc[dfd['tif4RCount'] != 8]), 0)

  def test_tiflocation12X(self):
    '''Test Location between tif1 and tif2 X'''    
    rdf = dfd['tif1X'].compare(dfd['tif2X'])       
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_tiflocation12Y(self):
    '''Test Location between tif1 and tif2 Y'''    
    rdf = dfd['tif1Y'].compare(dfd['tif2Y'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_tiflocation12Geo0(self):
    '''Test Location between tif1 and tif2 Geo0'''
    rdf = dfd['tif1Geo0'].compare(dfd['tif2Geo0'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_tiflocation12Geo3(self):
    '''Test Location between tif1 and tif2 Geo3'''
    rdf = dfd['tif1Geo3'].compare(dfd['tif2Geo3'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_tiflocation13X(self):
    '''Test Location between tif1 and tif3 X'''    
    rdf = dfd['tif1X'].compare(dfd['tif3X'])        
    self.assertEqual(rdf, self.emptyc, rdf.to_string())
  
  def test_tiflocation13Y(self):
    '''Test Location between tif1 and tif3 Y'''    
    rdf = dfd['tif1Y'].compare(dfd['tif3Y'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())
 
  def test_tiflocation13Geo0(self):
    '''Test Location between tif1 and tif3 Geo0'''
    rdf = dfd['tif1Geo0'].compare(dfd['tif3Geo0'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())
 
  def test_tiflocation13Geo3(self):
    '''Test Location between tif1 and tif3 Geo3'''
    rdf = dfd['tif1Geo3'].compare(dfd['tif3Geo3'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())
 
  def test_tiflocation45X(self):
    '''Test Location between tif4 and tif5 X'''    
    rdf = dfd['tif4X'].compare(dfd['tif5X'])        
    self.assertEqual(rdf, self.emptyc, rdf.to_string())
 
  def test_tiflocation45Y(self):
    '''Test Location between tif4 and tif5 Y'''    
    rdf = dfd['tif4Y'].compare(dfd['tif5Y'])        
    self.assertEqual(rdf, self.emptyc, rdf.to_string())
 
  def test_tiflocation45Geo0(self):
    '''Test Location between tif4 and tif5 Geo0'''    
    rdf = dfd['tif4Geo0'].compare(dfd['tif5Geo0'])        
    self.assertEqual(rdf, self.emptyc, rdf.to_string())
 
  def test_tiflocation45Geo3(self):
    '''Test Location between tif4 and tif5 Geo3'''    
    rdf = dfd['tif4Geo3'].compare(dfd['tif5Geo3'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())
 
  def test_firescarmin(self):
    '''Test Firescar tiff all min equals 0 '''  
    rdf = dfd.loc[dfd['tif1Min'] != 0]    
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_firescarmax(self):
    '''Test Firescar tiff all max equals 1'''      
    rdf = dfd.loc[dfd['tif1Max'] != 1] 
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())
  
  def test_severitymin(self):
    '''Test Severity tiff all min equals 0 '''  
    rdf = dfd.loc[dfd['tif2Min'] != 0]    
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_severitymax(self):
    '''Test Severity tiff all max between 1 and 3 '''  
    rdf = dfd.loc[(dfd['tif2Max'] < 1) | (dfd['tif2Max'] > 3)]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_firescar_xy_content(self):
    '''Test Firescar tiff x * y = zeros + ones '''    
    rdf1 = pd.DataFrame()
    rdf1['xy'] = dfd['tif1X']*dfd['tif1Y']
    rdf1['content'] = dfd['tif1Zeros']+dfd['tif1Ones']
    rdf = rdf1['xy'].compare(rdf1['content'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_severity_xy_content(self):
    '''Test Severity tiff x * y = zeros + ones + twos + threes '''    
    rdf1 = pd.DataFrame()
    rdf1['xy'] = dfd['tif2X']*dfd['tif2Y']
    rdf1['content'] = dfd['tif2Zeros']+dfd['tif2Ones']+dfd['tif2Twos']+dfd['tif2Threes']
    rdf = rdf1['xy'].compare(rdf1['content'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_firescar_severity_zeros(self):
    '''Test Firescar Zeros == Severity Zeros '''    
    rdf = dfd['tif1Zeros'].compare(dfd['tif2Zeros'])        
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_firescar_severity_nonzeros(self):
    '''Test Firescar NonZeros == Severity NonZeros ''' 
    rdf1 = pd.DataFrame()
    rdf1['content1'] = dfd['tif1Ones']
    rdf1['content2'] = dfd['tif2Ones']+dfd['tif2Twos']+dfd['tif2Threes']
    rdf = rdf1['content1'].compare(rdf1['content2'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string()) 

  def test_tif14X(self):
    '''Test if Firescar X dimensions equals Satellite image'''    
    rdf = dfd.loc[(dfd['tif1X'] == dfd['tif4X'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())    

  def test_tif14Y(self):
    '''Test if Firescar Y dimensions equals Satellite image'''
    rdf = dfd.loc[(dfd['tif1Y'] == dfd['tif4Y'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())



## Class Dbf Detailed Checks


In [ ]:
class testDbfDetails(testDF): 
  # Check Shapefiles: content, encoding, files
  def test_dbfFirescarSeverityPol(self):
    '''Test DBF Firescar and Severity: Polygon amount'''          
    rdf = dfd.loc[(dfd['lencolumnsShape1']<dfd['lencolumnsShape2'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_dbfFirescarSeverityRegion(self):
    '''Test DBF Firescar and Severity: Area Total'''    
    rdf = dfd['Region'].compare(dfd['Region.1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_dbfFirescarSeverity1(self):
    '''Test DBF Firescar and Severity: Area Total'''    
    rdf = dfd['AreaTotal'].compare(dfd['AreaTotal.1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_dbfFirescarSeverity2(self):
    '''Test DBF Firescar and Severity: Temporada'''    
    rdf = dfd['Temporada'].compare(dfd['Temporada.1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_dbfFirescarSeverity3(self):
    '''Test DBF Firescar and Severity: AreaINC_ha'''    
    rdf = dfd['AreaINC_ha'].compare(dfd['AreaINC_ha.1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_dbfFirescarSeverity4(self):
    '''Test DBF Firescar and Severity: AreaUnchS'''    
    rdf = dfd['AreaUnchS'].compare(dfd['AreaUnchS.1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_dbfFirescarSeverity5(self):
    '''Test DBF Firescar and Severity: AreaUnchS'''    
    rdf = dfd['AreaUnchS'].compare(dfd['AreaUnchS.1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_dbfFirescarSeverity6(self):
    '''Test DBF Firescar and Severity: AreaLowS'''    
    rdf = dfd['AreaLowS'].compare(dfd['AreaLowS.1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())
  
  def test_dbfFirescarSeverity7(self):
    '''Test DBF Firescar and Severity: AreaHighS'''    
    rdf = dfd['AreaHighS'].compare(dfd['AreaHighS.1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_dbfFirescarSeverity8(self):
    '''Test DBF Firescar and Severity: Control'''    
    rdf = dfd['Control'].compare(dfd['Control.1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_dbfFirescarSeverity9(self):
    '''Test DBF Firescar and Severity: AreaMedS'''    
    rdf = dfd['AreaMedS'].compare(dfd['AreaMedS.1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_dbfFirescarSeverity10(self):
    '''Test DBF Firescar and Severity: KEY'''    
    rdf = dfd['KEY'].compare(dfd['KEY.1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_dbfMixed1(self):
    '''Test DBF Firescar and Severity KEY FireId'''   
    rdf = dfd['IdKey'].compare(dfd.index.to_series())
    self.assertEqual(rdf, self.emptyc, rdf.to_string())




## Class Location Detailed Checks


In [ ]:
class testLocationDetails(testDF):
# dflocations=pd.merge(dfconafeval[[ 'Region', 'Cod_Region', 'Nombre inc', 'long', 'lat']], 
#dfd[['folderR_1', 'folderR_2', 'folderS_1', 'folderS_2', 'tif1Geo0', 'tif1Geo3', 'tif1Geo1', 'tif1Geo5', 'tif1X', 'tif1Y', 'Region', 'Region.1']],
#on='fireId', suffixes=['_conaf', '_dbf'])

  def test_region_master_conaf(self):
    '''Test Region between CONAF and master are equal'''    
    rdf = dfconafeval['Region'].compare(dfmeval['Region_CONAF'])        
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_region_dbfs(self):
    '''Test Region between CONAF and dbf are equal'''    
    rdf = dflocations['Region_dbf'].compare(dflocations['Region.1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_region_master_dbf(self):
    '''Test Region between CONAF and dbf are equal'''    
    rdf = dflocations['Region_conaf'].compare(dflocations['Region_dbf'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_region_dbf_folder1(self):
    '''Test Region between master and folderRegion are equal'''    
    rdf = dflocations['Region_conaf'].compare(dflocations['folderR_2'])        
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_region_dbf_folder2(self):
    '''Test Region between master and folderSeason are equal'''    
    rdf = dflocations['Region_conaf'].compare(dflocations['folderS_2'])        
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_region_folders(self):
    '''Test Region between folderRegion and folderSeason are equal'''    
    rdf = dflocations['folderR_2'].compare(dflocations['folderS_2'])        
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_region_folders2(self):
    '''Test Region between folderRegion and folderSeason are equal'''    
    rdf = dflocations['folderR_1'].compare(dflocations['folderS_1'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_region_folders3(self):
    '''Test Region between folderSeason and fireRegion are equal'''    
    rdf = dflocations['folderS_1'].compare(dflocations['fireRegion'])        
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_lat_master_conaf(self):
    '''Test Lat between CONAF and master are equal'''    
    rdf = dfconafeval['lat'].compare(dfmeval['lat'])        
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_lon_master_conaf(self):
    '''Test Lon between CONAF and master are equal'''    
    rdf = dfconafeval['long'].compare(dfmeval['long'])        
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_tif_location1(self):
    '''Test Firescar location in Satellite image LATITUDE (Geo3) Y  top''' 
    rdf = dfd.loc[(dfd['tif1Geo3'] > dfd['tif4Geo3'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_tif_location2(self):
    '''Test Firescar location in Satellite image LATITUDE (Geo3) Y bottom''' 
    rdf = dfd.loc[(dfd['tif1Geo3']+dfd['tif1Geo5']*dfd['tif1Y'] < dfd['tif4Geo3']+dfd['tif4Geo5']*dfd['tif4Y'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_tif_location3(self):
    '''Test Firescar location in Satellite image LONGITUDE (Geo3) X left''' 
    rdf = dfd.loc[(dfd['tif1Geo0'] < dfd['tif4Geo0'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_tif_location4(self):
    '''Test Firescar location in Satellite image LONGITUDE (Geo3) X right''' 
    rdf = dfd.loc[(dfd['tif1Geo0']+dfd['tif1Geo1']*dfd['tif1X'] > dfd['tif4Geo0']+dfd['tif4Geo1']*dfd['tif4X'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_tif_conaf1(self):
    '''Test Satellite image and CONAF LATITUDE top'''
    rdf = dfd.loc[(dfconafeval['lat'] > dfd['tif4Geo3'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_tif_conaf2(self):
    '''Test Satellite image and CONAF LATITUDE bottom'''
    rdf = dfd.loc[(dfconafeval['lat'] < dfd['tif4Geo3']+dfd['tif4Geo5']*dfd['tif4Y'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_tif_conaf3(self):
    '''Test Satellite image and CONAF LONGITUDE left'''
    rdf = dfd.loc[(dfconafeval['long'] < dfd['tif4Geo0'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_tif_conaf4(self):
    '''Test Satellite image and CONAF LONGITUDE right'''
    rdf = dfd.loc[(dfconafeval['long'] > dfd['tif4Geo0']+dfd['tif4Geo1']*dfd['tif4X'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())


  def test_tif_conaf1B(self):
    '''Test Firescar image and CONAF LATITUDE top'''
    rdf = dfd.loc[(dfconafeval['lat'] > dfd['tif1Geo3'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_tif_conaf2B(self):
    '''Test Firescar image and CONAF LATITUDE bottom'''
    rdf = dfd.loc[(dfconafeval['lat'] < dfd['tif1Geo3']+dfd['tif1Geo5']*dfd['tif1Y'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_tif_conaf3B(self):
    '''Test Firescar image and CONAF LONGITUDE left'''
    rdf = dfd.loc[(dfconafeval['long'] < dfd['tif1Geo0'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())

  def test_tif_conaf4B(self):
    '''Test Firescar image and CONAF LONGITUDE right'''
    rdf = dfd.loc[(dfconafeval['long'] > dfd['tif1Geo0']+dfd['tif1Geo1']*dfd['tif1X'])]
    self.assertEqual(rdf, self.emptydfd, rdf.to_string())




  def region_season_folder(self):
    '''Test REGION ID is equal for region folder and season folder Ex Fire_CL-AR_Araucania and FireScar_CL-AR_Araucania_1998 / CL-AR == CL-AR'''
    rdf = dfd['folderR_1'].compare(dfd['folderS_1']) 
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def region_season_name(self):
    '''Test REGION NAME is equal for region folder and season folder Ex Fire_CL-AR_Araucania and FireScar_CL-AR_Araucania_1998 /Araucania == Araucania'''
    rdf = dfd['folderR_2'].compare(dfd['folderS_2']) 
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def region_fire_region(self):
    '''Test REGION NAME IS DIFFERENT for region folder and fire filename region Ex Fire_CL-AR_Araucania and FireScar_CL-AR_ID151372_u390_20101211. / CL-AR == CL-AR'''
    rdf = dffires['folderR_2'].compare(dffires['fireRegion']) 
    self.assertEqual(rdf, self.emptyc, rdf.to_string())



In [ ]:
# Assert folderSeason year == get_season from filename date 

# WHEN CHECKING DATE!!
# filename FireScar_CL-BI_ID151372_u390_20101211.
# Ex FireScar_CL-AR_Araucania_1998 and 19981205 / 1998 == get_season(19981205)

#df['Diff'] = np.where( df['folderS_3'] == df['fireDate'].apply(get_season) , '1', '0')
#df2=df.loc[df['Diff'] == '0']
#df3=df2.groupby(['folderRegion', 'folderSeason', 'fireDate', 'fireId'], group_keys=False, as_index=False).agg(['count']).reset_index()
#with pd.ExcelWriter(output_path+'output_check_season.xlsx') as writer:  
#  df3.to_excel(writer, sheet_name='CheckSeason')

#if not df['folderS_3'].equals(df['fireDate'].apply(get_season)):
#  raise Exception("SEASON AND DATE IS DIFFERENT")

## Class Dates Season Detailed Checks

In [ ]:
class testDatesDetails(testDF):

  def test_date_master_conaf1(self):
    '''Test valid_start_date between CONAF and maestro are equal'''    
    rdf = dfconafeval['valid_start_date'].compare(dfmeval['valid_start_date'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_date_master_conaf2(self):
    '''Test valid_control_date between CONAF and maestro are equal'''    
    rdf = dfconafeval['valid_control_date'].compare(dfmeval['valid_control_date'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

  def test_date_master_fileDate(self):
    '''Test start date between maestro and filename are equal'''
    rdf = dffires['fireDate'].compare(dfmeval['stripped_start_date'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string()) 

  def test_season_folderSeason2(self):
    '''Test season between folderSeason and get_season(fireDate) are equal'''
    rdf = dffires['folderS_3'].compare(dflocations['season_from_date'])
    self.assertEqual(rdf, self.emptyc, str(len(rdf))) 

  def test_dbfFirescarSeverity2(self):
    '''Test Season between DBF and folderSeason Temporada'''    
    rdf = dfd['Temporada'].compare(dflocations['folderS_3'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())



In [ ]:
class testOverlapDetails(testDF):
  # REF dfi.rename(columns={'KEYID':'fireId'}, inplace=True)
  #dfi.set_index('fireId', inplace=True)
  # print('len dfi', len(dfi))
  # interaccion
  def test_overlaps_exist(self):
    '''Test all values in OverlapIDs are valid fireIDs'''    
    rdf = dfconafeval['valid_start_date'].compare(dfmeval['valid_start_date'])
    self.assertEqual(rdf, self.emptyc, rdf.to_string())

## Calls unit tests


### Tests testFileAmounts

In [ ]:
unittest.main(argv=[''], defaultTest='testFileAmounts', verbosity=3, exit=False)


test_details_vs_files (__main__.testFileAmounts)
Test Files in Details Obtained and NOT Filesystem ... ok
test_details_vs_files2 (__main__.testFileAmounts)
Test Files in Details Obtained and NOT Master ... ok
test_file_ext1 (__main__.testFileAmounts)
Test Fires with wrong amount of tif  (!= 5) ... ok
test_file_ext2 (__main__.testFileAmounts)
Test Fires with wrong amount of shape related files  (!= 2) ... ok
test_file_types1 (__main__.testFileAmounts)
Test Fires with wrong amount of FireScar files (!= 7) ... ok
test_file_types2 (__main__.testFileAmounts)
Test Fires with wrong amount of Severity files (!= 7) ... ok
test_file_types3 (__main__.testFileAmounts)
Test Fires with wrong amount of ImgPosF files (!= 1) ... ok
test_file_types4 (__main__.testFileAmounts)
Test Fires with wrong amount of ImgPreF (!= 1) ... ok
test_file_types5 (__main__.testFileAmounts)
Test Fires with wrong amount of RdNBR (!= 1) ... ok
test_fire_files_amount (__main__.testFileAmounts)
Test Number of Correct Files pe

124854876030


ok

----------------------------------------------------------------------
Ran 20 tests in 3.693s

OK


### Test testTifDetails

In [ ]:
unittest.main(argv=[''], defaultTest='testTifDetails', verbosity=2, exit=False)



test_area (__main__.testTifDetails)
Test area text ... ok
test_firescar_severity_nonzeros (__main__.testTifDetails)
Test Firescar NonZeros == Severity NonZeros ... ok
test_firescar_severity_zeros (__main__.testTifDetails)
Test Firescar Zeros == Severity Zeros ... ok
test_firescar_xy_content (__main__.testTifDetails)
Test Firescar tiff x * y = zeros + ones ... ok
test_firescarmax (__main__.testTifDetails)
Test Firescar tiff all max equals 1 ... ok
test_firescarmin (__main__.testTifDetails)
Test Firescar tiff all min equals 0 ... ok
test_projection (__main__.testTifDetails)
Test projection text ... ok
test_rastercount (__main__.testTifDetails)
Test Correct amount of rasters per tif ... ok
test_severity_xy_content (__main__.testTifDetails)
Test Severity tiff x * y = zeros + ones + twos + threes ... ok
test_severitymax (__main__.testTifDetails)
Test Severity tiff all max between 1 and 3 ... ok
test_severitymin (__main__.testTifDetails)
Test Severity tiff all min equals 0 ... ok
test_tif14X

### Test testDbfDetails

In [ ]:
unittest.main(argv=[''], defaultTest='testDbfDetails', verbosity=2, exit=False)


test_dbfFirescarSeverity1 (__main__.testDbfDetails)
Test DBF Firescar and Severity: Area Total ... ok
test_dbfFirescarSeverity10 (__main__.testDbfDetails)
Test DBF Firescar and Severity: KEY ... ok
test_dbfFirescarSeverity2 (__main__.testDbfDetails)
Test DBF Firescar and Severity: Temporada ... ok
test_dbfFirescarSeverity3 (__main__.testDbfDetails)
Test DBF Firescar and Severity: AreaINC_ha ... ok
test_dbfFirescarSeverity4 (__main__.testDbfDetails)
Test DBF Firescar and Severity: AreaUnchS ... ok
test_dbfFirescarSeverity5 (__main__.testDbfDetails)
Test DBF Firescar and Severity: AreaUnchS ... ok
test_dbfFirescarSeverity6 (__main__.testDbfDetails)
Test DBF Firescar and Severity: AreaLowS ... ok
test_dbfFirescarSeverity7 (__main__.testDbfDetails)
Test DBF Firescar and Severity: AreaHighS ... ok
test_dbfFirescarSeverity8 (__main__.testDbfDetails)
Test DBF Firescar and Severity: Control ... ok
test_dbfFirescarSeverity9 (__main__.testDbfDetails)
Test DBF Firescar and Severity: AreaMedS ... 

### Test testLocationDetails

In [ ]:
unittest.main(argv=[''], defaultTest='testLocationDetails', verbosity=3, exit=False)


test_lat_master_conaf (__main__.testLocationDetails)
Test Lat between CONAF and master are equal ... ok
test_lon_master_conaf (__main__.testLocationDetails)
Test Lon between CONAF and master are equal ... ok
test_region_dbf_folder1 (__main__.testLocationDetails)
Test Region between master and folderRegion are equal ... ok
test_region_dbf_folder2 (__main__.testLocationDetails)
Test Region between master and folderSeason are equal ... ok
test_region_dbfs (__main__.testLocationDetails)
Test Region between CONAF and dbf are equal ... ok
test_region_folders (__main__.testLocationDetails)
Test Region between folderRegion and folderSeason are equal ... ok
test_region_folders2 (__main__.testLocationDetails)
Test Region between folderRegion and folderSeason are equal ... ok
test_region_folders3 (__main__.testLocationDetails)
Test Region between folderSeason and fireRegion are equal ... ok
test_region_master_conaf (__main__.testLocationDetails)
Test Region between CONAF and master are equal ... 

### Test testDatesDetails

In [ ]:
unittest.main(argv=[''], defaultTest='testDatesDetails', verbosity=3, exit=False)


test_date_master_conaf1 (__main__.testDatesDetails)
Test valid_start_date between CONAF and maestro are equal ... ok
test_date_master_conaf2 (__main__.testDatesDetails)
Test valid_control_date between CONAF and maestro are equal ... ok
test_date_master_fileDate (__main__.testDatesDetails)
Test start date between maestro and filename are equal ... ok
test_dbfFirescarSeverity2 (__main__.testDatesDetails)
Test Season between DBF and folderSeason Temporada ... ok
test_season_folderSeason2 (__main__.testDatesDetails)
Test season between folderSeason and get_season(fireDate) are equal ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.215s

OK


# Generate Master

In [ ]:
  def test_print_Master(self):
    '''Printing Sizes report if needed'''
    if flag_print_reports:
      full_path=output_path+'output_statistics_size2021116_EXTRA.xlsx'
      with pd.ExcelWriter(full_path) as writer:  
        dfregions.to_excel(writer, sheet_name='SizePerRegion')
        dfseasons.to_excel(writer, sheet_name='SizePerSeason')
        dffires.to_excel(writer, sheet_name='SizePerFire')
      self.assertTrue(os.path.exists(full_path))
    else:
      pass


    # Check AREA from CONAF, maestro, tif, shape: area, zeros, ones, filesize
    # OK Check location from CONAF, maestro, tif -> get Region (&comuna?)
    # OK Check DATE from CONAF, maestro, shape
    
    # For each Fire: Get tif details, get shapefile details, compare with master and conaf in a huge excel by:
    # AREA,     LOCATION,      DATE 
    # AREA FROM FIRESCAR TIF (0 and 1)
    # OK

    # ARMAR MAESTRO CON MERGE, imprimir



#Take command-line input and do all this
getLocation(rlayer, -71.7793437883999,-36.2672338081154)
getLocation(rlayer, -71.79375,-36.28001)

'Nuble'